Let us start with the implementation of n-gram-based language model for the Opinion Spam Classification task.

Import Libaries and Open files

In [16]:
import pandas as pd
import numpy as np


#Open the Files

d_train = open("deceptive.txt")
t_train=open("truthful.txt")
d_val = open("deceptiveval.txt")
t_val = open("truthfulval.txt")
test=open("test.txt")



Get individual words from the file into lists

In [17]:
def individualtokens(file):
    allreview = file.readlines()
    reviews = []
    for word in allreview: 
         reviews.append(word.split(' '))
    return reviews


truth_train=individualtokens(t_train)
deceptive_train=individualtokens(d_train)
truth_val=individualtokens(t_val)
deceptive_val=individualtokens(d_val)
testdata=individualtokens(test)

Lowercase

In [18]:
def lowercase(data):
    newdata=[]
    for sentence in data:
         words = [word.lower() for word in sentence]
         newdata.append(words)
    return newdata    
            
            
      

updated_truth_train = lowercase(truth_train)
updated_deceptive_train=lowercase(deceptive_train)
updated_truth_val=lowercase(truth_val)
updated_deceptive_val=lowercase(deceptive_val)
updated_test=lowercase(testdata)


Unigrams and Unigram Counts

In [19]:
def Unigram(review):
    UnigramList={}
    for sentence in review:
        for word in sentence:
            if word not in UnigramList:
                UnigramList[word]=0
            UnigramList[word]+=1
    return UnigramList




truth_train_unigrams = Unigram(updated_truth_train)
deceptive_train_unigrams = Unigram(updated_deceptive_train)

#Counts

def UnigramCount(review):
    count=0
    for sentence in review:
        for word in sentence:
            count += 1
    return count




truth_train_Unigram_Count = UnigramCount(updated_truth_train)
deceptive_train_Unigram_Count= UnigramCount(updated_deceptive_train)

Bigrams and their counts

In [20]:
def Bigram(review):
    BigramList = {}
    for sentence in review: 
        combination = [(sentence[i], sentence[i + 1]) for i in range(len(sentence) - 1)]
        for  duo in combination:
            Bigram = duo[0] + " " + duo[1]
            if Bigram not in BigramList:
                BigramList[Bigram]=0
            BigramList[Bigram] += 1
    return BigramList   


            
truth_train_bigrams= Bigram(updated_truth_train)
deceptive_train_bigrams = Bigram(updated_deceptive_train) 

#Counts

def BigramCount(BigramList,review):
    count = sum(BigramList.values())
    return count
    
truth_train_Bigram_Count = BigramCount(truth_train_bigrams,updated_truth_train)
deceptive_train_Bigram_Count= BigramCount(deceptive_train_bigrams,updated_deceptive_train)



Unigram and Bigram Probabilities(Log Probabilities)

In [21]:
def UnigramProbability(Unigram,count):
    UnigramProbability={}
    for i in Unigram:
        UnigramProbability[i] = np.log(Unigram[i]/count)
    return UnigramProbability

    
truth_train_Unigram_Probability = UnigramProbability(truth_train_unigrams,truth_train_Unigram_Count)  
deceptive_train_Unigram_Probability = UnigramProbability(deceptive_train_unigrams,deceptive_train_Unigram_Count)


def BigramProbability(Bigram,Unigram):
    BigramProbability={}
    for i in Bigram:
        BigramProbability[i] = np.log(Bigram[i]/Unigram[i.split(" ")[0]])
    return BigramProbability    

truth_train_Bigram_Probability = BigramProbability(truth_train_bigrams,truth_train_unigrams)
deceptive_train_Bigram_Probability = BigramProbability(deceptive_train_bigrams,deceptive_train_unigrams)

Unique values in both the training sets for Unigrams and Bigrams

In [22]:
truth_train_Dict_count = len(truth_train_unigrams)
deceptive_train_Dict_count = len(deceptive_train_unigrams)
truth_train_Dict_bigram_count=len(truth_train_bigrams)
deceptive_train_Dict_bigram_count=len(deceptive_train_bigrams)

Unknown Word Handling

In [28]:
def UnknownWordHandler(review,unigram,n):   
    for review in sentence:
        for i in range(0,len(review)):
            if review[i] in unigram:
                if unigram[review[i]] < n:
                    review[i] = '<UNK>'
            if review[i] not in unigram:
                review[i] = '<UNK>'
                unigram[review[i]] += 1
    return unigram

Smoothing. After experimenting with values, we took k=1 for the test set and k=0.1 for the validation set. Also, we chose Unigram for smoothing.

In [24]:
def AdditiveSmoothingUnigram(review,Unigram,count,distinct,k):
    SmoothedUnigramProb={}
    for sentence in review:
        for word in sentence:
            if word in Unigram.keys():
                SmoothedUnigramProb[word] = np.log((k+ Unigram[word])/(count+(k*distinct)))
            else:
                SmoothedUnigramProb[word] = np.log(k/(count+(k*distinct)))
    return SmoothedUnigramProb   


#On Truthful Validation Set
truth_val_Unigrams_Probability = AdditiveSmoothingUnigram(updated_truth_val,truth_train_unigrams,truth_train_Unigram_Count,truth_train_Dict_count,0.1)
deceptive_val_Unigrams_Probability = AdditiveSmoothingUnigram(updated_truth_val,deceptive_train_unigrams,deceptive_train_Unigram_Count,deceptive_train_Dict_count,0.1)

#On Deceptive Validation Set
truth_val_Unigrams_Probability1 = AdditiveSmoothingUnigram(updated_deceptive_val,truth_train_unigrams,truth_train_Unigram_Count,truth_train_Dict_count,0.1)
deceptive_val_Unigrams_Probability1 = AdditiveSmoothingUnigram(updated_deceptive_val,deceptive_train_unigrams,deceptive_train_Unigram_Count,deceptive_train_Dict_count,0.1)

#On Test Set
truth_test_Unigrams_Probabilty=AdditiveSmoothingUnigram(updated_test,truth_train_unigrams,truth_train_Unigram_Count,truth_train_Dict_count,1)
deceptive_test_Unigrams_Probability = AdditiveSmoothingUnigram(updated_test,deceptive_train_unigrams,deceptive_train_Unigram_Count,deceptive_train_Dict_count,1)


Perplexity

In [25]:
def UnigramProbabilityforPerplexity(review,dictionary):
    total = 0
    for word in review:
        total = total + dictionary[word]
    return total    
        
  

def Perplexity(review,smootheddictionary,n_gram_probability):
    return np.exp((-1/len(review))*n_gram_probability(review,smootheddictionary))

Now let us compute the accuracies for both the validation sets based on the Perplexity. We return the lower perplexity class.

In [26]:
#Initializing Dict to store values of class after perplexity calculation
dict_truthful_output={}
dict_deceptive_output={}

#Truthful Validation Set
for i in range(0,len(updated_truth_val)):
    if Perplexity(updated_truth_val[i],truth_val_Unigrams_Probability,UnigramProbabilityforPerplexity) < Perplexity(updated_truth_val[i],deceptive_val_Unigrams_Probability,UnigramProbabilityforPerplexity):
        dict_truthful_output[i] = 1
    else:
        dict_truthful_output[i] = 0
  
    


Accuracy_truth_Unigram_Val = (sum(dict_truthful_output.values())/len(updated_truth_val))

print('Truth Validation Set Accuracy: ' + str(Accuracy_truth_Unigram_Val*100))     


#Deceptive Validation Set

for i in range(0,len(updated_deceptive_val)):
    if Perplexity(updated_deceptive_val[i],truth_val_Unigrams_Probability1,UnigramProbabilityforPerplexity) > Perplexity(updated_deceptive_val[i],deceptive_val_Unigrams_Probability1,UnigramProbabilityforPerplexity):
        dict_deceptive_output[i] = 1
    else:
         dict_deceptive_output[i]=0
        
  
    
Accuracy_deceptive_Unigram_Val = (sum(dict_deceptive_output.values())/len(updated_deceptive_val))
print('Deceptive Validation Set Accuracy: ' + str(Accuracy_deceptive_Unigram_Val*100))  



Truth Validation Set Accuracy: 94.53125
Deceptive Validation Set Accuracy: 92.96875


Now let's go ahead on our test set :)

In [27]:
dict_test_output={}

for i in range(0,len(updated_test)):
    if Perplexity(updated_test[i],truth_test_Unigrams_Probabilty,UnigramProbabilityforPerplexity) < Perplexity(updated_test[i],deceptive_test_Unigrams_Probability,UnigramProbabilityforPerplexity):
       dict_test_output[i] = 0
       print(str(i) + ',' + '0')
    else:
        dict_test_output[i] = 1
        print(str(i) + ',' + '1')

0,0
1,0
2,1
3,1
4,0
5,0
6,1
7,1
8,1
9,0
10,1
11,1
12,1
13,1
14,1
15,1
16,1
17,0
18,1
19,0
20,0
21,1
22,1
23,1
24,1
25,1
26,1
27,1
28,1
29,0
30,1
31,0
32,0
33,0
34,1
35,0
36,1
37,0
38,1
39,1
40,0
41,1
42,1
43,1
44,1
45,1
46,0
47,1
48,1
49,1
50,0
51,1
52,0
53,1
54,1
55,1
56,0
57,1
58,0
59,1
60,1
61,1
62,0
63,1
64,0
65,1
66,0
67,0
68,0
69,1
70,1
71,1
72,0
73,0
74,0
75,1
76,1
77,1
78,1
79,1
80,0
81,0
82,0
83,0
84,0
85,0
86,0
87,1
88,0
89,0
90,1
91,0
92,0
93,0
94,1
95,1
96,1
97,1
98,1
99,0
100,0
101,0
102,0
103,1
104,0
105,0
106,0
107,1
108,1
109,1
110,0
111,1
112,0
113,1
114,1
115,0
116,0
117,1
118,1
119,1
120,0
121,0
122,1
123,0
124,0
125,1
126,0
127,1
128,0
129,1
130,0
131,0
132,1
133,1
134,0
135,0
136,0
137,1
138,0
139,1
140,0
141,1
142,0
143,0
144,1
145,1
146,0
147,1
148,1
149,0
150,0
151,1
152,0
153,1
154,1
155,1
156,1
157,1
158,1
159,1
160,0
161,0
162,1
163,0
164,0
165,1
166,1
167,1
168,1
169,0
170,1
171,1
172,0
173,0
174,0
175,1
176,1
177,0
178,1
179,0
180,1
181,1
182,0
183,1
184,1
